In [1]:
# Data must have each day fully populated with 24*4 demand data and sequentially labelled
# It is assumed that the earliest entry of the training data starts with midnight, and that null entries represent 0 demand

import pandas as pd
import os
import Geohash
import numpy as np
import pickle
import time
import xgboost as xgb
import lightgbm as lgb



DATA_PATH = os.getcwd() + "/Data/training.csv"

# XGBoost params
max_depth = 6
n_estimators = 1000
max_bins = 2048

# LGB params
lgb_params = {
    'lambda_l2':1,
    'metric': 'rmse',
    'objective': 'regression',
    'device_type':'cpu',
    'boosting_type': 'gbdt',
    'num_leaves':40,
    'min_data_in_leaf':50,
    'seed' : 1337,
    'max_bin': 2048,
    'min_split_gain': 0.001,
    'subsample_for_bin': 5000,

}


In [2]:
def parse_raw_data(data_path=DATA_PATH):
    """Parses csv file into numpy array"""
    df = pd.read_csv(data_path)

    # Add latt and long columns
    df['latt'] = df['geohash6'].apply(lambda x: Geohash.decode(x)[0])
    df['long'] = df['geohash6'].apply(lambda x: Geohash.decode(x)[1])

    # Extract unique latt and long values
    latt_vals = sorted(df['latt'].unique())
    long_vals = sorted(df['long'].unique())

    latt_to_idx_dic = {latt:idx for idx,latt in enumerate(latt_vals)}
    long_to_idx_dic = {long:idx for idx,long in enumerate(long_vals)}

    # Save dic
    with open("Model/latt_to_idx_dic.pkl","wb") as f:
        pickle.dump(latt_to_idx_dic,f)


    with open("Model/long_to_idx_dic.pkl","wb") as f:
        pickle.dump(long_to_idx_dic,f)

    # Populate raw demand data onto np array of shape (day_time_periods,latt,long)
    demand_data = []
    for day in sorted(df['day'].unique()):
        filtered_day = df[df['day'] == day]
        print("Day %d"%(day),end='\r')
        for hour in range(24):
            for minute in "0 15 30 45".split():
                timestamp = "%d:%s"%(hour,minute)
                a = np.zeros((len(latt_vals),len(long_vals)))
                filtered_time = filtered_day[filtered_day['timestamp']==timestamp]

                for idx,item in filtered_time[['latt','long','demand']].iterrows():
                    latt,long,demand = item
                    latt_idx = latt_to_idx_dic[latt]
                    long_idx = long_to_idx_dic[long]
                    a[latt_idx,long_idx] = demand
                demand_data.append(a)
    return np.stack(demand_data,axis=0)


In [3]:
def extract_features(data):
    """
    Extracts features from numpy array of demand, for training
    Numpy array should start with midnight for the first day and contain data for at least 8 days

    Parameters
    ----------
    data : Numpy array
        3D Numpy array of demand data, with the shape (n_periods, n_latt, n_long), where
            n_periods refers to the number of 15-minute periods
            n_latt refers to the number of lattitude values
            n_long refers to the number of longitude values

    Returns
    -------
    Numpy Array
        4D Numpy array of features, with the shape (n_features, n_feature_periods, n_latt, n_long), where
            n_features refers to the number of features for each data sample
            n_feature_periods refers to the number of 15-minute periods with features
            n_latt refers to the number of lattitude values
            n_long refers to the number of longitude values
    """
    day_periods = 24*4
    k = 0.0001 # Smoothener for normalizations with potentially 0 divisors
    all_features = []
    all_meta_features = {}
    n_periods = data.shape[0]
    # Normal Features
    # Current day features
    for X in [-7,-6,-5,-4,-3,-2,-1]:
        all_features.append(data[7*day_periods+X+3:X+1+n_periods,:,:])
    # Previous day features
    for D in [-7,-4,-3,-2,-1]:
        for X in [-3,-2,-1,0,1,2,3,4]:
            all_features.append(data[(7+D)*day_periods + X+3:D*day_periods + X+1+n_periods,:,:])
            

    # Time period features
    day_periods_arr = np.arange(day_periods)/day_periods
    
    sin_arr = np.sin(2*np.pi*day_periods_arr)
    sin_arr = np.tile(sin_arr,int(n_periods/24/4)+int(n_periods%(24*4)>0))
    sin_arr = sin_arr[7*day_periods + 3:]
    all_features.append(sin_arr[:,None,None]*np.ones((1,1,data.shape[2]))*np.ones((1,data.shape[1],1)))
    
    cos_arr = np.cos(2*np.pi*day_periods_arr)
    cos_arr = np.tile(cos_arr,int(n_periods/24/4)+int(n_periods%(24*4)>0))
    cos_arr = cos_arr[7*day_periods + 3:]
    all_features.append(cos_arr[:,None,None]*np.ones((1,1,data.shape[2]))*np.ones((1,data.shape[1],1)))
    
    # Weekday period features
    weekday_periods_arr = np.arange(7)/7
    
    sin_arr = np.sin(2*np.pi*weekday_periods_arr)
    sin_arr = np.repeat(sin_arr,day_periods)
    sin_arr = np.tile(sin_arr,int(n_periods/24/4/7) + 1)
    sin_arr = sin_arr[7*day_periods + 3:-(24*4*7 - n_periods%(24*4*7) - 1)]
    all_features.append(sin_arr[:,None,None]*np.ones((1,1,data.shape[2]))*np.ones((1,data.shape[1],1)))
    
    cos_arr = np.cos(2*np.pi*weekday_periods_arr)
    cos_arr = np.repeat(cos_arr,day_periods)
    cos_arr = np.tile(cos_arr,int(n_periods/24/4/7) + 1)
    cos_arr = cos_arr[7*day_periods + 3:-(24*4*7 - n_periods%(24*4*7) - 1)]
    all_features.append(cos_arr[:,None,None]*np.ones((1,1,data.shape[2]))*np.ones((1,data.shape[1],1)))
    
    
    
    # Geospatial features
    latt = (np.arange(data.shape[1])[:,None]/data.shape[1]) * np.ones(data.shape[2])[None,:]
    long = np.ones(data.shape[1])[:,None] * (np.arange(data.shape[2])[None,:]/data.shape[2])
    all_features.extend([
        np.ones(n_periods - 7*day_periods-2)[:,None,None]*item[None,:,:]
        for item in (latt,long)
    ])


    # Aggregrated demand features by location
    agg_arr = np.sum(data,axis=0,keepdims=True) + k
    agg_arr = agg_arr/np.max(agg_arr)
    all_features.append(agg_arr*np.ones((n_periods - 7*day_periods-2,1,1)))
    all_meta_features['agg_location'] = agg_arr
    
    
    # Aggregrated demand features by time period
    full_agg_arr = np.sum(np.sum(data,axis=1,keepdims=True),axis=2,keepdims=True).squeeze()
    divisor = full_agg_arr[:(full_agg_arr.shape[0]//96)*96].reshape((-1,96))
    divisor = np.max(divisor,axis=0)
    all_meta_features['agg_period'] = divisor
    divisor = np.tile(divisor, reps=full_agg_arr.shape[0]//96+1)
    divisor = divisor[:full_agg_arr.shape[0]]
    full_agg_arr /= divisor
    # Current day aggregrated demand
    for X in [-4,-3,-2,-1]:
        agg_arr = full_agg_arr[7*day_periods+X+3:X+1+n_periods,None,None]
        all_features.append(agg_arr*np.ones((1,data.shape[1],data.shape[2])))
    # Past day aggregrated demand
    for D in [-7]:
        for X in [-1,0,1]:
            agg_arr = full_agg_arr[(7+D)*day_periods + X+3:D*day_periods + X+1+n_periods,None,None]
            all_features.append(agg_arr*np.ones((1,data.shape[1],data.shape[2])))
    
    # Save normalizers
    with open("Model/all_meta_features.pkl","wb") as f:
        pickle.dump(all_meta_features,f)
    return np.stack(all_features,axis=0)



def format_features_labels(features, targets):
    """
    Formats features and labels into an appropriate shape for training

    Parameters
    ----------
    features : Numpy array
        4D Numpy array of demand data, with the shape (n_features, n_feature_periods, n_latt, n_long), where
            n_features refers to the number of features for each data sample
            n_feature_periods refers to the number of 15-minute periods with features
            n_latt refers to the number of lattitude values
            n_long refers to the number of longitude values
    targets : Numpy array
        3D Numpy array of demand data, with the shape (n_feature_periods, n_latt, n_long), where
            n_periods refers to the number of 15-minute periods with features
            n_latt refers to the number of lattitude values
            n_long refers to the number of longitude values

    Returns
    -------
    Numpy Array
        2D Numpy array of data features, with the shape (n_features, n_samples), where
            n_features refers to the number of features for each data sample
            n_samples refers to the number of data samples for training
    Numpy Array
        1D Numpy array of data labels, with the shape (n_samples), where
            n_samples refers to the number of data samples for training
    """
    features = features.reshape((features.shape[0],-1)).T
    targets = targets.reshape(-1)
    
    return features,targets


def train_lgb(train_X, train_y, params, model=None,max_round=500):
    """
    Trains a LGB with the specified params. If a model is provided, it will continue training that model.

    Parameters
    ----------
    train_X : Numpy Array
        2D Numpy array of data features, with the shape (n_features, n_samples), where
            n_features refers to the number of features for each data sample
            n_samples refers to the number of data samples for training
    train_y : Numpy Array
        1D Numpy array of data labels, with the shape (n_samples), where
            n_samples refers to the number of data samples for training
    params : Dictionary
        Dictionary of all parameters for training LightGBM
    model : Booster
        LightGBM Booster instance representing a trained model, or None to start training from scratch
    max_round : int
        Maximum number of rounds for training the model

    Returns
    -------
    Booster
        The trained LightGBM Booster model
    """
    train_ds = lgb.Dataset(train_X, label=train_y)

    model = lgb.train(
        params, 
        train_ds, 
        num_boost_round=max_round,
        valid_sets=[train_ds],
        verbose_eval=50,
        early_stopping_rounds=20,
        init_model=model
    )
    return model

def train_model(demand_data):
    """
    Extracts features from demand_data and fully trains XGBoost model with demand data
    Trained model will be saved to /Model folder

    Parameters
    ----------
    demand_data : Numpy array
        3D Numpy array of demand data, with the shape (n_periods, n_latt, n_long), where
            n_periods refers to the number of 15-minute periods
            n_latt refers to the number of lattitude values
            n_long refers to the number of longitude values
    """
    # Extract features for XGBoost
    day_periods = 24*4
    train_y = demand_data[day_periods*7 + 3:,:,:]
    train_X = demand_data[:-1,:,:]
    nonzero_demand = np.sum(demand_data,axis=0)
    np.save("Model/zero_demand.npy",np.nonzero((nonzero_demand<=0)))
    nonzero_demand = np.nonzero((nonzero_demand>0))
    
    train_X = extract_features(train_X)
    train_X = train_X[:,:,nonzero_demand[0],nonzero_demand[1]]
    train_y = train_y[:,nonzero_demand[0],nonzero_demand[1]]
    train_X,train_y = format_features_labels(train_X,train_y)
    del nonzero_demand, demand_data
    
    
    # Train models and save
    start = time.time()
    # Train LGB Model
    lgbm = None
    for lr,max_round in zip((0.1,0.05),(1000,500)):
        lgb_params['learning_rate'] = lr
        lgbm = train_lgb(train_X,train_y,params=lgb_params,model=lgbm,max_round=max_round)
    # Train XGBRegressor
    gbm = xgb.XGBRegressor(
        tree_method='gpu_hist',
        max_bin=max_bins,
        max_depth=max_depth, 
        n_estimators=n_estimators, 
        learning_rate=0.05,
        objective='reg:squarederror'
    ).fit(train_X, train_y)
    
    
    end = time.time()
    
    print("Minutes: %.4f"%((end-start)/60))


    
    # Save model to file
    pickle.dump(gbm, open("Model/xgb_model.model", "wb"))
    pickle.dump(lgbm, open("Model/lgb_model.model", "wb"))

In [4]:
def run():
    demand_data = parse_raw_data()
    train_model(demand_data)


In [5]:
if __name__ == "__main__":
    run()

Training until validation scores don't improve for 20 rounds.
[50]	training's rmse: 0.0226132
[100]	training's rmse: 0.0221741
[150]	training's rmse: 0.0219492
[200]	training's rmse: 0.0217981
[250]	training's rmse: 0.0216807
[300]	training's rmse: 0.0215812
[350]	training's rmse: 0.0214924
[400]	training's rmse: 0.0214108
[450]	training's rmse: 0.0213388
[500]	training's rmse: 0.0212744
[550]	training's rmse: 0.0212139
[600]	training's rmse: 0.0211577
[650]	training's rmse: 0.0211009
[700]	training's rmse: 0.0210482
[750]	training's rmse: 0.0209978
[800]	training's rmse: 0.0209483
[850]	training's rmse: 0.0209029
[900]	training's rmse: 0.0208574
[950]	training's rmse: 0.0208148
[1000]	training's rmse: 0.0207773
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.0207773
Training until validation scores don't improve for 20 rounds.
[1050]	training's rmse: 0.0207578
[1100]	training's rmse: 0.0207395
[1150]	training's rmse: 0.0207196
[1200]	training's rmse: 0.020701